In [76]:
from serial import Serial
import pandas as pd
import seaborn as sns
from pyubx2 import UBXReader, UBX_PROTOCOL, UBXMessage, SET_LAYER_RAM, POLL_LAYER_RAM, TXN_COMMIT, TXN_NONE

In [2]:
usb_tty_port = '/dev/cu.usbmodem1201'

In [10]:
# Poll configuration of "CFG_MSGOUT_UBX_TIM_TP_USB". On startup, should be 0 by default.
layer = POLL_LAYER_RAM
position = 0
keys = ["CFG_MSGOUT_UBX_TIM_TP_USB"]

msg = UBXMessage.config_poll(layer, position, keys)
print(msg)
with Serial(usb_tty_port, 38400, timeout=3) as stream:
    stream.write(msg.serialize())
    ubr_poll_status = UBXReader(stream, protfilter=UBX_PROTOCOL)
    raw_data, parsed_data = ubr_poll_status.read()
    if parsed_data is not None:
        print(parsed_data)

<UBX(CFG-VALGET, version=0, layer=0, position=0, keys_01=546374016)>
<UBX(CFG-VALGET, version=1, layer=0, position=0, CFG_MSGOUT_UBX_TIM_TP_USB=0)>


In [74]:
# Set configuration of "CFG_MSGOUT_UBX_TIM_TP_USB" to 1.
layer = SET_LAYER_RAM
transaction = TXN_NONE
cfgData = [("CFG_MSGOUT_UBX_TIM_TP_USB", 1)]

msg = UBXMessage.config_set(layer, transaction, cfgData)
print(msg)
with Serial(usb_tty_port, 38400, timeout=100) as stream:
    stream.write(msg.serialize())
    ubr = UBXReader(stream, protfilter=UBX_PROTOCOL)
    for i in range(2):
        raw_data, parsed_data = ubr.read()
        if parsed_data is not None:
            print(parsed_data)

<UBX(CFG-VALSET, version=0, ram=1, bbr=0, flash=0, action=0, reserved0=0, CFG_MSGOUT_UBX_TIM_TP_USB=1)>
<UBX(ACK-ACK, clsID=CFG, msgID=CFG-VALSET)>
<UBX(TIM-TP, towMS=510511000, towSubMS=0.0, qErr=764, week=2319, timeBase=0, utc=1, raim=2, qErrInvalid=0, TpNotLocked=0, timeRefGnss=0, utcStandard=0)>


In [71]:
# Read UBX packets from the receiver via the usb_tty_port and write the TIM-TP packets to the pd.DataFrame tim_tp_df
tim_tp_df = pd.DataFrame(
    columns=['payload',
             'towMS (ms)', # Byte offset 4: towMS (unit: ms)
             'towSubMS', # Byte offset 8: towSubMS (unit: ms, scale: 2^-32)
             'qErr (ps)', # Byte offset 8: qErr (unit: ps)
             'week (weeks)', # Byte offset 12: week (unit: weeks)
             # Byte offset 14: flags
             'timeBase_flag',
             'utc_flag',
             'raim_flag',
             'qErrInvalid_flag',
             # Byte offset 15: refInfo
             'timeRefGnss'
            ]
)

with Serial(usb_tty_port, 38400, timeout=3) as stream:
    ubr = UBXReader(stream, protfilter=UBX_PROTOCOL)
    try:
        while True:
            raw_data, parsed_data = ubr.read()
            if parsed_data and parsed_data.identity == 'TIM-TP':
                print(parsed_data)
                # See section 3.19.2.1 UBX-TIM-TP
                data = {
                    'payload': parsed_data.payload,
                    'towMS (ms)': parsed_data.towMS, 
                    'towSubMS': parsed_data.towSubMS, 
                    'qErr (ps)': parsed_data.qErr, 
                    'week (weeks)': parsed_data.week, 
                    'timeBase_flag': parsed_data.timeBase,
                    'utc_flag': parsed_data.utc,
                    'raim_flag': parsed_data.raim,
                    'qErrInvalid_flag': parsed_data.qErrInvalid,
                    'timeRefGnss': parsed_data.timeRefGnss,
                }
                tim_tp_df.loc[len(tim_tp_df)] = data
                # print(data)
    except KeyboardInterrupt:
        pass

<UBX(TIM-TP, towMS=510412000, towSubMS=0.0, qErr=1414, week=2319, timeBase=0, utc=1, raim=2, qErrInvalid=0, TpNotLocked=0, timeRefGnss=0, utcStandard=0)>
<UBX(TIM-TP, towMS=510413000, towSubMS=0.0, qErr=2350, week=2319, timeBase=0, utc=1, raim=2, qErrInvalid=0, TpNotLocked=0, timeRefGnss=0, utcStandard=0)>
<UBX(TIM-TP, towMS=510414000, towSubMS=0.0, qErr=3095, week=2319, timeBase=0, utc=1, raim=2, qErrInvalid=0, TpNotLocked=0, timeRefGnss=0, utcStandard=0)>
<UBX(TIM-TP, towMS=510415000, towSubMS=0.0, qErr=3705, week=2319, timeBase=0, utc=1, raim=2, qErrInvalid=0, TpNotLocked=0, timeRefGnss=0, utcStandard=0)>
<UBX(TIM-TP, towMS=510416000, towSubMS=0.0, qErr=-3671, week=2319, timeBase=0, utc=1, raim=2, qErrInvalid=0, TpNotLocked=0, timeRefGnss=0, utcStandard=0)>
<UBX(TIM-TP, towMS=510417000, towSubMS=0.0, qErr=-3240, week=2319, timeBase=0, utc=1, raim=2, qErrInvalid=0, TpNotLocked=0, timeRefGnss=0, utcStandard=0)>
<UBX(TIM-TP, towMS=510418000, towSubMS=0.0, qErr=-1817, week=2319, timeBas

In [73]:
tim_tp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 0 to 10
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   payload           11 non-null     object 
 1   towMS (ms)        11 non-null     int64  
 2   towSubMS          11 non-null     float64
 3   qErr (ps)         11 non-null     int64  
 4   week (weeks)      11 non-null     int64  
 5   timeBase_flag     11 non-null     int64  
 6   utc_flag          11 non-null     int64  
 7   raim_flag         11 non-null     int64  
 8   qErrInvalid_flag  11 non-null     int64  
 9   timeRefGnss       11 non-null     int64  
dtypes: float64(1), int64(8), object(1)
memory usage: 968.0+ bytes


In [75]:
sns.scatterplot

,payload,towMS (ms),towSubMS,qErr (ps),week (weeks),timeBase_flag,utc_flag,raim_flag,qErrInvalid_flag,timeRefGnss
0,b'\xe0Dl\x1e\x00\x00\x00\x00\x86\x05\x00\x00\x...,510412000,0.0,1414,2319,0,1,2,0,0
1,b'\xc8Hl\x1e\x00\x00\x00\x00.\t\x00\x00\x0f\t\...,510413000,0.0,2350,2319,0,1,2,0,0
2,b'\xb0Ll\x1e\x00\x00\x00\x00\x17\x0c\x00\x00\x...,510414000,0.0,3095,2319,0,1,2,0,0
3,b'\x98Pl\x1e\x00\x00\x00\x00y\x0e\x00\x00\x0f\...,510415000,0.0,3705,2319,0,1,2,0,0
4,b'\x80Tl\x1e\x00\x00\x00\x00\xa9\xf1\xff\xff\x...,510416000,0.0,-3671,2319,0,1,2,0,0
5,b'hXl\x1e\x00\x00\x00\x00X\xf3\xff\xff\x0f\t\n...,510417000,0.0,-3240,2319,0,1,2,0,0
6,b'P\\l\x1e\x00\x00\x00\x00\xe7\xf8\xff\xff\x0f...,510418000,0.0,-1817,2319,0,1,2,0,0
7,b'8`l\x1e\x00\x00\x00\x00c\xfb\xff\xff\x0f\t\n...,510419000,0.0,-1181,2319,0,1,2,0,0
8,b' dl\x1e\x00\x00\x00\x00z\xfc\xff\xff\x0f\t\n...,510420000,0.0,-902,2319,0,1,2,0,0
9,b'\x08hl\x1e\x00\x00\x00\x00\x87\xf8\xff\xff\x...,510421000,0.0,-1913,2319,0,1,2,0,0


In [62]:
d = [attr for attr in dir(parsed_data) if attr[0] != '_']
remove_attrs = ['TpNotLocked', 'config_del', 'config_poll', 'config_set', 'serialize']
for attr in remove_attrs:
    d.remove(attr)
print(d)
parsed_data.week

['identity', 'length', 'msg_cls', 'msg_id', 'msgmode', 'payload', 'qErr', 'qErrInvalid', 'raim', 'timeBase', 'timeRefGnss', 'towMS', 'towSubMS', 'utc', 'utcStandard', 'week']


2319